# Orchestrator Agent

This notebook orchestrates the entire ASRA workflow by coordinating the different agents and managing the research process.

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from typing import Dict, List
import json
from datetime import datetime
from utils.config import setup_logging, OUTPUTS_DIR
from utils.helpers import save_json
import traceback

In [ ]:
# Setup logging
logger = setup_logging('orchestrator')

class ResearchOrchestrator:
    def __init__(self):
        self.agents_dir = Path.cwd()
        self.workflow_status = {}
        
    def execute_notebook(self, notebook_path: Path) -> Dict:
        """Execute a Jupyter notebook and return its output."""
        try:
            # Load notebook
            with open(notebook_path) as f:
                nb = nbformat.read(f, as_version=4)
            
            # Execute notebook
            ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
            ep.preprocess(nb, {'metadata': {'path': str(notebook_path.parent)}})
            
            return {
                'status': 'success',
                'notebook': notebook_path.name
            }
            
        except Exception as e:
            logger.error(f"Error executing notebook {notebook_path}: {str(e)}")
            return {
                'status': 'error',
                'notebook': notebook_path.name,
                'error': str(e),
                'traceback': traceback.format_exc()
            }
    
    def run_research_workflow(self) -> Dict:
        """Execute the complete research workflow."""
        workflow_steps = [
            ('literature_review.ipynb', 'Literature Review'),
            ('hypothesis_generator.ipynb', 'Hypothesis Generation'),
            ('data_analyzer.ipynb', 'Data Analysis'),
            ('visualizer.ipynb', 'Visualization')
        ]
        
        results = {}
        for notebook, step_name in workflow_steps:
            logger.info(f'Starting {step_name}')
            
            # Execute notebook
            notebook_path = self.agents_dir / notebook
            step_result = self.execute_notebook(notebook_path)
            
            # Store results
            results[step_name] = step_result
            
            # Check for errors
            if step_result['status'] == 'error':
                logger.error(f'Workflow failed at {step_name}')
                break
            
            logger.info(f'Completed {step_name}')
        
        # Save workflow results
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = OUTPUTS_DIR / f'workflow_results_{timestamp}.json'
        save_json(results, output_path)
        
        return results

In [ ]:
def run_research_pipeline() -> None:
    """Run the complete research pipeline."""
    logger.info('Starting research pipeline')
    
    try:
        # Initialize orchestrator
        orchestrator = ResearchOrchestrator()
        
        # Run workflow
        results = orchestrator.run_research_workflow()
        
        # Check overall status
        success = all(step['status'] == 'success' for step in results.values())
        
        if success:
            logger.info('Research pipeline completed successfully')
        else:
            logger.error('Research pipeline completed with errors')
        
        return results
        
    except Exception as e:
        logger.error(f'Error in research pipeline: {str(e)}')
        return None

In [ ]:
if __name__ == "__main__":
    # Run pipeline
    results = run_research_pipeline()
    
    # Print summary
    if results:
        print("\nWorkflow Summary:")
        for step, result in results.items():
            status = "✓" if result['status'] == 'success' else "✗"
            print(f"{status} {step}")
            if result['status'] == 'error':
                print(f"  Error: {result['error']}")